This notebook is following the following tutorial:
https://www.kaggle.com/sanikamal/text-classification-with-python-and-keras#Table-of-Contents

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.
/Users/thomascartwright/miniconda3/envs/sci/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


# Build a Text Classifier

### Load project

In [3]:
projects = pd.read_pickle("../Data/project_data")

In [4]:
projects.head()

,UN_regions,country,description,error,guid,origin,regions,tags,title,topics,url,project_id
0,[],,Foldit is a revolutionary new computer game en...,NaN,5f80760d-8398-5091-b3c6-f34c39216e88,scistarter,[],"[dna, protein]",Foldit,"[Computers & Technology, Biology, Chemistry]",https://scistarter.com/project/4-Foldit-Foldit,4
1,[],,Firefly Watch combines an annual summer evenin...,NaN,c9664e0c-819a-5a42-b4bb-5f25d83a486d,scistarter,"[{'geometry': {'type': 'MultiPolygon', 'coordi...","[boston, cambridge, fireflies, insects, lightn...",Firefly Watch,"[Nature & Outdoors, Animals, Ecology & Environ...",https://scistarter.com/project/5-Firefly-Watch...,5
2,[],,Galaxy Zoo needs your help to classify galaxie...,NaN,11f65e99-b463-5e01-ac11-ae59c021dfe7,scistarter,[],"[astronomy & space, space, space science, zoon...",Galaxy Zoo,"[Astronomy & Space, Computers & Technology]",https://scistarter.com/project/6-Galaxy-Zoo-Ga...,6
3,[],None,Pay attention to the plants and animals in you...,NaN,wilsoncenter:27-107,scistarter,"[{'geometry': {'type': 'MultiPolygon', 'coordi...","[android, animal, animals, app, biology, clima...",Nature's Notebook,"[Ocean, Water, Marine & Terrestrial, Education...",https://scistarter.com/project/7-Natures-Noteb...,7
4,[],,A recent issue of Make magazine (http://makezi...,NaN,ae91e967-6eec-5aef-ab3a-7d86ceff737a,scistarter,[],[],Laser Harp: Build It Yourself,"[Computers & Technology, Sound, Physics]",https://scistarter.com/project/8-Laser-Harp-Bu...,8


In [9]:
titles = projects['title']

### Vectorize the sentences

In [10]:
vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(titles)
vectorizer.vocabulary_

{'Foldit': 895,
 'Firefly': 870,
 'Watch': 2483,
 'Galaxy': 946,
 'Zoo': 2584,
 'Nature': 1565,
 'Notebook': 1609,
 'Laser': 1273,
 'Harp': 1052,
 'Build': 359,
 'It': 1192,
 'Yourself': 2575,
 'North': 1603,
 'American': 134,
 'Amphibian': 136,
 'Monitoring': 1497,
 'Program': 1835,
 'Radio': 1877,
 'JOVE': 1194,
 'Cure': 610,
 'Together': 2333,
 'Roadkill': 1972,
 'Project': 1837,
 'Wildlife': 2536,
 'CoCoRaHS': 514,
 'Rain': 1879,
 'Hail': 1043,
 'Snow': 2149,
 'Network': 1582,
 'EnvironMentors': 784,
 'Earthdive': 741,
 'Casey': 431,
 'Trees': 2373,
 'Butterflies': 366,
 've': 2907,
 'Seen': 2081,
 'Incredible': 1143,
 'Crayon': 586,
 'Physics': 1753,
 'Spider': 2185,
 'WebWatch': 2499,
 'Fish': 874,
 'Watchers': 2485,
 'ReefBase': 1909,
 'BioBlitz': 280,
 'Portable': 1810,
 'Antiquities': 154,
 'Scheme': 2043,
 'Citizen': 491,
 'Weather': 2497,
 'Observer': 1626,
 'Santa': 2035,
 'Fe': 847,
 'National': 1561,
 'Forest': 902,
 'Site': 2125,
 'Stewards': 2222,
 'Squirrel': 2202,
 'S

In [11]:
vectorizer.transform(titles).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Defining a Baseline Model

<b> <font color='red'>I stopped this here because the tutorial then went on to do SUPERVISED learning which I am not doing</font> </b>

The tutorial then goes on to do one-hot encodings but this doesnt make sense for the titles or descriptions

### Word Embedding

In [12]:
# Setup a tokenizer which will accept the 5000 most common words
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(titles)

In [17]:
X = tokenizer.texts_to_sequences(titles)

vocab_size = len(tokenizer.word_index) + 1 # Adding 1 because of reserved 0 in dex

print(titles.iloc[4])
print(X[4])

Laser Harp: Build It Yourself
[876, 877, 878, 80, 879]


In [19]:
for word in ['the', 'all','it']:
    print('{}: {}'.format(word, tokenizer.word_index[word]))

the: 1
all: 436
it: 80
